# Issue 8

Issue 8 shows some artifacts in the horizon calculation. This seems to step from the `hor1d` function not looking at the adjacent pixel or not searching the full length of the vector.

In [ ]:
import numpy as np
import rasterio as rio
import pandas as pd
import matplotlib.pyplot as plt

from topocalc.horizon import horizon

In [ ]:
ras = rio.open('data/test_proj_windowed_int.tif')
ar = ras.read(1)
ar = ar.astype(np.float)

plt.figure(figsize=(15,15))
plt.imshow(ar)
plt.colorbar()

In [ ]:
horizon_test_azimuth_0 = np.rad2deg(np.arccos(horizon(0, ar, 30)))

plt.figure(figsize=(15,15))
plt.imshow(horizon_test_azimuth_0)
plt.colorbar()

In [ ]:
# Pull out into 2D, look at this particular column that has a 90 degree horizon
idx = 519
surf = ar[:, idx]
hor = horizon(0, ar, 30)[:, idx]

df = pd.DataFrame([surf, hor], index=['surface', 'horizon']).transpose()

fig,ax = plt.subplots(figsize=(20,10))
df.surface.plot(ax=ax)
df.horizon.plot(ax=ax, secondary_y=True)
ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]], df.columns)

In [ ]:
# surface
N = len(df.surface)
dx = 30
m = np.repeat(df.surface.values.reshape(1,-1), N, axis=0)

# distance to each point
# k=-1 because the distance to the point itself is 0
distance = dx * np.cumsum(np.tri(N, N, k=-1), axis=0)

# height change
height = np.tril(m.T - m)

# slope
slope = height / distance

# horizon location
hor = np.nanargmax(slope[:, :-1], axis=0)
hor = np.append(hor, N-1)
df['horizon_index'] = hor

fig, ax = plt.subplots(2, 2, figsize=(10,10))
ax = ax.flatten()
ax[0].imshow(m)
ax[0].set_title('surface height')

ax[1].imshow(distance)
ax[1].set_title('distance from point to all other points')

ax[2].imshow(height)
ax[2].set_title('height difference')

ax[3].imshow(slope)
ax[3].set_title('slope to all points')

In [ ]:
# turn the horizon locations into cosine

df['horizon_height'] = surf[hor]
df['distance'] = dx * (hor - np.arange(0, N))

df['new_horizon'] =  df.horizon_height / np.sqrt(df.horizon_height**2 + df.distance**2)

fig,ax = plt.subplots(2, 1, figsize=(20,10))
ax = ax.flatten()
df[['surface', 'horizon_height']].plot(ax=ax[0])

np.rad2deg(np.arccos(df[['horizon', 'new_horizon']])).plot(ax=ax[1])
